In [7]:
import pandas as pd
import numpy as np
import tensorflow as tf
import seaborn as sns
import pyCompare
import matplotlib
import keras_tuner as kt
import h5py


from sklearn.preprocessing import StandardScaler, MinMaxScaler, OneHotEncoder, Normalizer, RobustScaler, MaxAbsScaler
from yellowbrick.regressor import PredictionError, ResidualsPlot
from matplotlib import pyplot as plt
from scipy.signal import savgol_filter
from sklearn.metrics import r2_score, mean_squared_error
from math import sqrt
from keras_tuner.tuners import BayesianOptimization, Hyperband
from tensorflow.keras.utils import plot_model
from tensorflow.keras.layers import LeakyReLU, Input, Dense, Dropout, Flatten, Conv1D, MaxPooling1D, concatenate
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras import regularizers
from tensorflow.keras.metrics import MeanSquaredError, MeanAbsoluteError
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from scipy.ndimage import uniform_filter1d, gaussian_filter1d, median_filter
import pywt
from sklearn.decomposition import PCA
import sys
from io import StringIO


# Set seeds for reproducibility
np.random.seed(42)
tf.random.set_seed(42)

sns.set(rc={'figure.figsize': (10, 6)})
sns.set(style="whitegrid", font_scale=2)

def read_and_clean_data(data_source, drop_outliers=True):
    if isinstance(data_source, str):
        data = pd.read_csv(data_source)
    elif isinstance(data_source, StringIO):
        data = pd.read_csv(data_source)
    else:
        raise ValueError("data_source must be a file path (str) or a StringIO object.")
    
    if drop_outliers:
        data = data[data['sample'] != 'outlier']
    
    data.rename(columns={"final_age": "Age"}, inplace=True)
    
    if data.isnull().values.any():
        raise ValueError("Data contains NaN values")

    print(data.head())
    print(list(data.columns[0:10]))  # ['file_name', 'sample', 'Age', 'latitude', 'length', 'gear_depth', 'gear_temp', 'wn11476.85064', 'wn11468.60577', 'wn11460.36091']
    return data

# Savitzky-Golay filter function
def savgol_filter_func(data, window_length=17, polyorder=2, deriv=1):
    return savgol_filter(data, window_length=window_length, polyorder=polyorder, deriv=deriv)

# Moving Average filter function
def moving_average_filter(data, size=5):
    return uniform_filter1d(data, size=size, axis=1)

# Gaussian filter function
def gaussian_filter_func(data, sigma=2):
    return gaussian_filter1d(data, sigma=sigma, axis=1)

# Median filter function
def median_filter_func(data, size=5):
    return median_filter(data, size=(1, size))

# Wavelet filter function
def wavelet_filter_func(data, wavelet='db1', level=1):
    def apply_wavelet(signal):
        coeffs = pywt.wavedec(signal, wavelet, level=level)
        coeffs[1:] = [pywt.threshold(i, value=0.5 * max(i)) for i in coeffs[1:]]
        return pywt.waverec(coeffs, wavelet)
    
    return np.apply_along_axis(apply_wavelet, axis=1, arr=data)

# Fourier filter function
def fourier_filter_func(data, threshold=0.1):
    def apply_fft(signal):
        fft_data = np.fft.fft(signal)
        frequencies = np.fft.fftfreq(len(signal))
        fft_data[np.abs(frequencies) > threshold] = 0
        return np.fft.ifft(fft_data).real
    
    return np.apply_along_axis(apply_fft, axis=1, arr=data)

# PCA filter function
def pca_filter_func(data, n_components=5):
    pca = PCA(n_components=n_components)
    transformed = pca.fit_transform(data)
    return pca.inverse_transform(transformed)

# Main preprocessing function
def preprocess_spectra(data, filter_type='savgol'):
    filter_functions = {
        'savgol': savgol_filter_func,
        'moving_average': moving_average_filter,
        'gaussian': gaussian_filter_func,
        'median': median_filter_func,
        'wavelet': wavelet_filter_func,
        'fourier': fourier_filter_func,
        'pca': pca_filter_func
    }
    
    filter_func = filter_functions.get(filter_type, savgol_filter_func)
    
    data.loc[data['sample'] == 'training', data.columns[4:]] = filter_func(data.loc[data['sample'] == 'training', data.columns[4:]].values)
    data.loc[data['sample'] == 'test', data.columns[4:]] = filter_func(data.loc[data['sample'] == 'test', data.columns[4:]].values)
    
    return data

def apply_normalization(data, columns):
    normalizer = Normalizer()
    data[columns] = normalizer.fit_transform(data[columns])
    return data

def apply_robust_scaling(data, y_col, feature_columns):
    scaler_y = RobustScaler()
    data[y_col] = scaler_y.fit_transform(data[[y_col]])
    data[feature_columns] = data[feature_columns].apply(lambda col: RobustScaler().fit_transform(col.values.reshape(-1, 1)))
    return data, scaler_y

def apply_minmax_scaling(data, y_col, feature_columns):
    scaler_y = MinMaxScaler()
    data[y_col] = scaler_y.fit_transform(data[[y_col]])
    data[feature_columns] = data[feature_columns].apply(lambda col: MinMaxScaler().fit_transform(col.values.reshape(-1, 1)))
    return data, scaler_y

def apply_maxabs_scaling(data, y_col, feature_columns):
    scaler_y = MaxAbsScaler()
    data[y_col] = scaler_y.fit_transform(data[[y_col]])
    data[feature_columns] = data[feature_columns].apply(lambda col: MaxAbsScaler().fit_transform(col.values.reshape(-1, 1)))
    return data, scaler_y

def apply_scaling(data, scaling_method='standard', y_col='Age'):
    scalers = {
        'standard': StandardScaler(),
        'minmax': MinMaxScaler(),
        'maxabs': MaxAbsScaler(),
        'robust': RobustScaler(),
        'normalize': Normalizer()  
    }
    
    if scaling_method not in scalers:
        raise ValueError(f"Unsupported scaling method: {scaling_method}")
    
    feature_columns = data.columns.difference(['sample', 'file_name', y_col])

    if scaling_method == 'normalize':
        data = apply_normalization(data, feature_columns)
        return data, None  
    
    scaler_y = scalers[scaling_method]
    data[y_col] = scaler_y.fit_transform(data[[y_col]])
    scaler_x = scalers[scaling_method]
    data[feature_columns] = scaler_x.fit_transform(data[feature_columns])
    
    return data, scaler_y

def build_model(hp, input_dim_A, input_dim_B):
    input_A = Input(shape=(input_dim_A,))
    x = input_A

    input_B = Input(shape=(input_dim_B, 1))
    
    # Define the hyperparameters
    num_conv_layers = hp.Int('num_conv_layers', 1, 4, default=1)
    kernel_size = hp.Int('kernel_size', 51, 201, step=10, default=101)
    stride_size = hp.Int('stride_size', 26, 101, step=5, default=51)
    dropout_rate = hp.Float('dropout_rate', 0.1, 0.5, step=0.05, default=0.1)
    use_max_pooling = hp.Boolean('use_max_pooling', default=False)
    num_filters = hp.Int('num_filters', 50, 100, step=10, default=50)

    y = input_B
    for i in range(num_conv_layers):
        y = Conv1D(
            filters=num_filters,
            kernel_size=kernel_size,
            strides=stride_size,
            activation='relu',
            padding='same')(y)
        
        # Ensure the input size is appropriate for max pooling
        if use_max_pooling and y.shape[1] > 1:
            y = MaxPooling1D(pool_size=2)(y)
        
        y = Dropout(dropout_rate)(y)

    y = Flatten()(y)
    y = Dense(4, activation="relu", name='output_B')(y)

    con = concatenate([x, y])

    z = Dense(
        hp.Int('dense', 4, 640, step=32, default=256),
        activation='relu')(con)
    z = Dropout(hp.Float('dropout-2', 0.0, 0.5, step=0.05, default=0.0))(z)

    output = Dense(1, activation="linear")(z)
    model = Model(inputs=[input_A, input_B], outputs=output)
    model.compile(optimizer='adam', loss='mse', metrics=['mse', 'mae'])
    return model

def train_and_optimize_model(tuner, data, nb_epoch, batch_size):
    outputFilePath = 'Estimator'
    checkpointer = ModelCheckpoint(filepath=outputFilePath, verbose=1, save_best_only=True)
    earlystop = EarlyStopping(monitor='val_loss', patience=7, verbose=1, restore_best_weights=True)

    X_train_biological_data = data.loc[data['sample'] == 'training', data.columns[4:8]]
    X_train_wavenumbers = data.loc[data['sample'] == 'training', data.columns[8:]]
    y_train = data.loc[data['sample'] == 'training', 'Age']

    tuner.search([X_train_biological_data, X_train_wavenumbers], y_train,
                 epochs=nb_epoch,
                 batch_size=batch_size,
                 shuffle=True,
                 validation_split=0.25,
                 verbose=1,
                 callbacks=[earlystop])

    model = tuner.get_best_models(num_models=1)[0]
    best_hp = tuner.get_best_hyperparameters()[0]

    return model, best_hp

def final_training_pass(model, data, nb_epoch, batch_size):
    outputFilePath = 'Estimator'
    checkpointer = ModelCheckpoint(filepath=outputFilePath, verbose=1, save_best_only=True)
    earlystop = EarlyStopping(monitor='val_loss', patience=100, verbose=1, restore_best_weights=True)

    X_train_biological_data = data.loc[data['sample'] == 'training', data.columns[4:8]]
    X_train_wavenumbers = data.loc[data['sample'] == 'training', data.columns[8:]]
    y_train = data.loc[data['sample'] == 'training', 'Age']

    history = model.fit([X_train_biological_data, X_train_wavenumbers], y_train,
                        epochs=nb_epoch,
                        batch_size=batch_size,
                        shuffle=True,
                        validation_split=0.25,
                        verbose=1,
                        callbacks=[checkpointer, earlystop]).history
    
    return history

def plot_training_history(history):
    plt.figure(figsize=(10, 10))
    plt.plot(history['loss'])
    plt.plot(history['val_loss'])
    plt.title('Model Loss')
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Validation'], loc='upper right')
    plt.show()

def evaluate_model(model, data):
    X_test_biological_data = data.loc[data['sample'] == 'test', data.columns[4:8]]
    X_test_wavenumbers = data.loc[data['sample'] == 'test', data.columns[8:]]
    y_test = data.loc[data['sample'] == 'test', 'Age']

    evaluation = model.evaluate([X_test_biological_data, X_test_wavenumbers], y_test)
    preds = model.predict([X_test_biological_data, X_test_wavenumbers])
    r2 = r2_score(y_test, preds)
    return evaluation, preds, r2

def plot_predictions(y_test, preds):
    plt.figure(figsize=(6, 6))
    plt.scatter(y_test, preds)
    plt.xlabel('True')
    plt.ylabel('Predicted')
    lims = [-2.5, 5]
    plt.xlim(lims)
    plt.ylim(lims)
    plt.plot(lims, lims)
    plt.show()

def plot_prediction_error(preds, y_test):
    preds = np.array(preds).flatten()
    y_test = y_test.to_numpy().flatten()
    error = preds - y_test

    plt.figure(figsize=(6, 6))
    plt.hist(error, bins=20)
    plt.xlabel('Prediction Error')
    plt.ylabel('Count')
    plt.show()

def evaluate_training_set(model, data, scaler_y):
    X_train_biological_data = data.loc[data['sample'] == 'training', data.columns[4:8]]
    X_train_wavenumbers = data.loc[data['sample'] == 'training', data.columns[8:]]
    y_train = data.loc[data['sample'] == 'training', 'Age']
    f_train = data.loc[data['sample'] == 'training', 'file_name']

    y_train = np.array(y_train).reshape(-1, 1)
    preds_t = model.predict([X_train_biological_data, X_train_wavenumbers])
    
    preds_t = preds_t.reshape(-1, 1)
    y_train_reshaped = y_train.reshape(-1, 1)
    
    y_pr_transformed = scaler_y.inverse_transform(preds_t)
    y_tr_transformed = scaler_y.inverse_transform(y_train_reshaped)

    r_squared_tr = r2_score(y_tr_transformed, y_pr_transformed)
    rmse_tr = sqrt(mean_squared_error(y_tr_transformed, y_pr_transformed))

    y_tr_df = pd.DataFrame(y_tr_transformed, columns=['train'])
    y_tr_df['pred'] = y_pr_transformed
    y_tr_df['file'] = f_train.reset_index(drop=True)

    y_tr_df.to_csv('./Output/Data/train_predictions.csv', index=False)

    return r_squared_tr, rmse_tr, y_tr_df

def plot_training_set(y_tr_transformed, y_pr_transformed):
    sns.set_style("white")
    sns.set(style="ticks")
    sns.set_context("poster")

    f, ax = plt.subplots(figsize=(12, 12))
    p = sns.regplot(x=y_tr_transformed, y=y_pr_transformed, ci=None,
                    scatter_kws={"edgecolor": 'b', 'linewidths': 2, "alpha": 0.5, "s": 150},
                    line_kws={"alpha": 0.5, "lw": 4})
    ax.plot([y_tr_transformed.min(), y_tr_transformed.max()], [y_tr_transformed.min(), y_tr_transformed.max()], 'k--', lw=2)

    p.set(xlim=(-1, 24))
    p.set(ylim=(-1, 24))
    sns.despine()
    plt.title('Training Set', fontsize=25)
    plt.xlabel('Traditional Age (years)')
    plt.ylabel('FT-NIR Age (years)')
    plt.savefig('./Output/Figures/TrainingSet.png')
    plt.show()

def plot_test_set(y_test_transformed, y_pred_transformed):
    f, ax = plt.subplots(figsize=(12, 12))
    p = sns.regplot(x=y_test_transformed, y=y_pred_transformed, ci=None,
                    scatter_kws={"edgecolor": 'b', 'linewidths': 2, "alpha": 0.5, "s": 150},
                    line_kws={"alpha": 0.5, "lw": 4})
    ax.plot([y_test_transformed.min(), y_test_transformed.max()], [y_test_transformed.min(), y_test_transformed.max()], 'k--', lw=2)

    p.set(xlim=(-1, 24))
    p.set(ylim=(-1, 24))
    sns.despine()
    plt.title('Test Set', fontsize=25)
    plt.xlabel('Traditional Age (years)')
    plt.ylabel('FT-NIR Age (years)')
    plt.savefig('./Output/Figures/TestSet.png')
    plt.show()

def plot_bland_altman(y_test_transformed, y_pred_transformed):
    pyCompare.blandAltman(y_test_transformed.flatten(), y_pred_transformed.flatten(),
                          limitOfAgreement=1.96, confidenceInterval=95,
                          confidenceIntervalMethod='approximate',
                          detrend=None, percentage=False,
                          title='Bland-Altman Plot\n',
                          savePath='./Output/Figures/BlandAltman.png')

def build_model_manual(input_dim_A, input_dim_B, num_conv_layers, kernel_size, stride_size, dropout_rate, use_max_pooling, num_filters, dense_units, dropout_rate_2):
    input_A = Input(shape=(input_dim_A,))
    x = input_A

    input_B = Input(shape=(input_dim_B, 1))
    y = input_B
    for i in range(num_conv_layers):
        y = Conv1D(
            filters=num_filters,
            kernel_size=kernel_size,
            strides=stride_size,
            activation='relu',
            padding='same')(y)
        
        if use_max_pooling and y.shape[1] > 1:
            y = MaxPooling1D(pool_size=2)(y)
        
        y = Dropout(dropout_rate)(y)

    y = Flatten()(y)
    y = Dense(4, activation="relu", name='output_B')(y)

    con = concatenate([x, y])

    z = Dense(dense_units, activation='relu')(con)
    z = Dropout(dropout_rate_2)(z)

    output = Dense(1, activation="linear")(z)
    model = Model(inputs=[input_A, input_B], outputs=output)
    model.compile(optimizer='adam', loss='mse', metrics=['mse', 'mae'])
    return model

def InferenceMode(model_or_path, data, row_number, scaler_y=None, scaler_x=None):
    if isinstance(model_or_path, str):
        # Load model from disk
        model = load_model(model_or_path)
    else:
        # Use the provided model object
        model = model_or_path
    
    # Extract the specific row for inference
    sample_data = data.iloc[row_number]
    
    # Extract biological and wavenumber data separately
    biological_data = sample_data[data.columns[4:8]].values.reshape(1, -1)
    wavenumber_data = sample_data[data.columns[8:]].values.reshape(1, -1, 1)
    
    # Apply the same scaling used during training
    if scaler_x:
        biological_data = scaler_x.transform(biological_data)
        wavenumber_data = scaler_x.transform(wavenumber_data.reshape(1, -1)).reshape(1, -1, 1)
    
    # Run inference
    prediction = model.predict([biological_data, wavenumber_data])
    
    # Inverse transform the prediction to original scale
    if scaler_y:
        prediction = scaler_y.inverse_transform(prediction)
    
    return prediction

def TrainingModeWithHyperband(filepath, filter_CHOICE, scaling_CHOICE):
    data = read_and_clean_data(filepath)
    
    data = preprocess_spectra(data, filter_type=filter_CHOICE)
    
    y_col = 'Age'
    scaling_method = scaling_CHOICE  # 'minmax', 'standard', 'maxabs', 'robust', or 'normalize'
    data, scaler_y = apply_scaling(data, scaling_method, y_col)

    input_dim_A = data.columns[4:8].shape[0]
    input_dim_B = data.columns[8:].shape[0]
    
    def model_builder(hp):
        return build_model(hp, input_dim_A, input_dim_B)
    
    tuner = Hyperband(
        model_builder,
        objective='val_loss',
        max_epochs=1,
        directory='Tuners',
        project_name='mmcnn',
        seed=42
    )
    
    print(tuner.search_space_summary())
    
    nb_epoch = 1  # !@!
    batch_size = 32
    model, best_hp = train_and_optimize_model(tuner, data, nb_epoch, batch_size)
    
    nb_epoch = 1  # !@!
    batch_size = 32
    history = final_training_pass(model, data, nb_epoch, batch_size)
    
    evaluation, preds, r2 = evaluate_model(model, data)
    print(f"Evaluation: {evaluation}, R2: {r2}")
    
    model.summary()
    
    training_outputs = {
        'trained_model': model,
        'scaler' : scaler_y,
        'training_history': history,
        'evaluation': evaluation,
        'predictions': preds,
        'r2_score': r2
    }
    
    # top_5_models = tuner.get_best_models(num_models=5)
    
    additional_outputs = {
        # 'top5models': top_5_models
    }
    
    return training_outputs, additional_outputs

def TrainingModeWithoutHyperband(filepath, filter_CHOICE, scaling_CHOICE, model_parameters):
    if len(model_parameters) != 8:
        raise ValueError("model_parameters must be a list of 8 values.")
    
    num_conv_layers, kernel_size, stride_size, dropout_rate, use_max_pooling, num_filters, dense_units, dropout_rate_2 = model_parameters
    
    if not all(isinstance(param, (int, float, bool)) for param in model_parameters):
        raise ValueError("All model parameters must be either int, float, or bool.")
    
    data = read_and_clean_data(filepath)
    
    data = preprocess_spectra(data, filter_type=filter_CHOICE)
    
    y_col = 'Age'
    scaling_method = scaling_CHOICE  # 'minmax', 'standard', 'maxabs', 'robust', or 'normalize'
    data, scaler_y = apply_scaling(data, scaling_method, y_col)

    input_dim_A = data.columns[4:8].shape[0]
    input_dim_B = data.columns[8:].shape[0]

    model = build_model_manual(
        input_dim_A,
        input_dim_B,
        num_conv_layers,
        kernel_size,
        stride_size,
        dropout_rate,
        use_max_pooling,
        num_filters,
        dense_units,
        dropout_rate_2
    )
    
    nb_epoch = 1  # !@!
    batch_size = 32
    history = final_training_pass(model, data, nb_epoch, batch_size)
    
    evaluation, preds, r2 = evaluate_model(model, data)
    print(f"Evaluation: {evaluation}, R2: {r2}")
    
    model.summary()
    
    training_outputs = {
        'trained_model': model,
        'scaler': scaler_y,
        'training_history': history,
        'evaluation': evaluation,
        'predictions': preds,
        'r2_score': r2
    }
    
    additional_outputs = {
        
    }
    
    return training_outputs, additional_outputs


def saveModel(model, path):
    model.save(path)
    print(f"Model saved to {path}")


def saveModelWithMetadata(model, path, column_names, description):

    # Save the Keras model
    model.save(path)
    print(f"Model saved to {path}")

    # Append metadata to the HDF5 file
    with h5py.File(path, 'a') as f:
        # Create a group for metadata if it doesn't exist
        if 'metadata' not in f:
            metadata_group = f.create_group('metadata')
        else:
            metadata_group = f['metadata']

        # Add the column names list
        if 'column_names' in metadata_group:
            del metadata_group['column_names']  # Delete if exists to avoid duplication
        metadata_group.create_dataset('column_names', data=column_names)
        
        # Add the description string
        if 'description' in metadata_group:
            del metadata_group['description']  # Delete if exists to avoid duplication
        metadata_group.create_dataset('description', data=description)
        
    print(f"Metadata added to {path}")

def loadModelMetadata(path):

    with h5py.File(path, 'r') as f:
        metadata_group = f['metadata']
        column_names = list(metadata_group['column_names'])
        description = metadata_group['description'][()].decode('utf-8')

    return column_names, description


In [33]:
import random

In [34]:
def create_dataframe():
    num_rows = 100

    data = {
        'filename': [f'file_{i}' for i in range(num_rows)],
        'sample': ['training' if random.random() < 0.8 else 'test' for i in range(num_rows)],
        'age': np.random.randint(1, 100, num_rows),
        'weight': np.random.uniform(1.0, 100.0, num_rows),
        'length': np.random.uniform(1.0, 100.0, num_rows),
        'latitude': np.random.uniform(-90.0, 90.0, num_rows),
        'longitude': np.random.uniform(-180.0, 180.0, num_rows),
        'sex_M': np.random.choice([0, 1], num_rows),
        'sex_F': np.random.choice([0, 1], num_rows),
        'sex_immature': np.random.choice([0, 1], num_rows), 
    }

    for i in range(1, 91):
        data[f'var_{i}'] = np.random.randint(-1, 10, num_rows)

    for i in range(1, 1001):
        data[f'wavenumber_{i}'] = np.random.uniform(-1.0, 1.0, num_rows)

    # data['metadata'] = [4000, 9000, 5] + [np.nan] * (num_rows - 3)

    df = pd.DataFrame(data)
    return df

sample_data_format = create_dataframe()

sample_data_format

sample_data_format.to_csv('Data/sample_data.csv', index=False)





# filename, sample, age, weight, length, latitude, longitude, sex_M, sex_F, sex_immature, var_1-90,
# Wavenumbers: columns 101+ i.e. 101-1100
# Last column: wavenumber metadata: start, end, step (rows 1-3) not anymore!



In [5]:
# Training With Hyperband
training_outputs_hyperband, additional_outputs_hyperband = TrainingModeWithHyperband(
    filepath='./Data/AGP_MMCNN_BSsurvey_pollock2014to2018.csv',
    filter_CHOICE='savgol',
    scaling_CHOICE='minmax'
)

print(training_outputs_hyperband)
print(additional_outputs_hyperband)

                                  file_name    sample   Age  latitude  length  \
0  WALLEYE_POLLOCK_157201603201_60411_OT1.0  training  10.0  53.97653   610.0   
1  WALLEYE_POLLOCK_157201603201_60412_OT1.0  training   9.0  53.97653   611.0   
2  WALLEYE_POLLOCK_157201603201_60414_OT1.0  training   6.0  53.97653   443.0   
3  WALLEYE_POLLOCK_157201603201_60428_OT1.0  training   8.0  53.97653   479.0   
4  WALLEYE_POLLOCK_157201603201_60450_OT1.0  training   6.0  53.97653   451.0   

   gear_depth  gear_temp  wn11476.85064  wn11468.60577  wn11460.36091  ...  \
0      407.26       4.33       0.313066       0.313311       0.313633  ...   
1      407.26       4.33       0.329684       0.330087       0.330593  ...   
2      407.26       4.33       0.320618       0.321387       0.321874  ...   
3      407.26       4.33       0.323654       0.323983       0.324420  ...   
4      407.26       4.33       0.321214       0.321375       0.321607  ...   

   wn4072.962798  wn4064.717934  wn4056.4730

INFO:tensorflow:Assets written to: Estimator\assets


55/55 [==============================] - 0s 2ms/step
Evaluation: [0.02767881192266941, 0.02767881192266941, 0.13219918310642242], R2: -0.35598078096565033
Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_2 (InputLayer)        [(None, 907, 1)]             0         []                            
                                                                                                  
 conv1d (Conv1D)             (None, 15, 70)               7140      ['input_2[0][0]']             
                                                                                                  
 dropout (Dropout)           (None, 15, 70)               0         ['conv1d[0][0]']              
                                                                                                  
 flatten (Flatten)           (None, 10

In [8]:
# Training Without Hyperband

training_outputs_manual, additional_outputs_manual = TrainingModeWithoutHyperband(
    filepath='./Data/AGP_MMCNN_BSsurvey_pollock2014to2018.csv',
    filter_CHOICE='savgol',
    scaling_CHOICE='minmax',
    model_parameters=[2, 101, 51, 0.1, False, 50, 256, 0.1]
)

# num_conv_layers, kernel_size, stride_size, dropout_rate, use_max_pooling, num_filters, dense_units, dropout_rate_2 


print(training_outputs_manual)
print(additional_outputs_manual)

                                  file_name    sample   Age  latitude  length  \
0  WALLEYE_POLLOCK_157201603201_60411_OT1.0  training  10.0  53.97653   610.0   
1  WALLEYE_POLLOCK_157201603201_60412_OT1.0  training   9.0  53.97653   611.0   
2  WALLEYE_POLLOCK_157201603201_60414_OT1.0  training   6.0  53.97653   443.0   
3  WALLEYE_POLLOCK_157201603201_60428_OT1.0  training   8.0  53.97653   479.0   
4  WALLEYE_POLLOCK_157201603201_60450_OT1.0  training   6.0  53.97653   451.0   

   gear_depth  gear_temp  wn11476.85064  wn11468.60577  wn11460.36091  ...  \
0      407.26       4.33       0.313066       0.313311       0.313633  ...   
1      407.26       4.33       0.329684       0.330087       0.330593  ...   
2      407.26       4.33       0.320618       0.321387       0.321874  ...   
3      407.26       4.33       0.323654       0.323983       0.324420  ...   
4      407.26       4.33       0.321214       0.321375       0.321607  ...   

   wn4072.962798  wn4064.717934  wn4056.4730

INFO:tensorflow:Assets written to: Estimator\assets


55/55 [==============================] - 0s 3ms/step
Evaluation: [0.004343908745795488, 0.004343908745795488, 0.0469832606613636], R2: 0.7871925515974116
Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_4 (InputLayer)        [(None, 907, 1)]             0         []                            
                                                                                                  
 conv1d_1 (Conv1D)           (None, 18, 50)               5100      ['input_4[0][0]']             
                                                                                                  
 dropout_2 (Dropout)         (None, 18, 50)               0         ['conv1d_1[0][0]']            
                                                                                                  
 conv1d_2 (Conv1D)           (None, 1

In [9]:
# Saving Model

model = training_outputs_hyperband['trained_model']
saveModel(model, "my_model.h5")

Model saved to my_model.h5


C:\Users\Michael.Zakariaie\AppData\Roaming\Python\Python311\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [10]:
# Saving Model w/ Metadata

column_names = ['col1', 'col2', 'col3']
description = 'This model is trained on data columns col1, col2, col3'
saveModelWithMetadata(model, 'my_model_with_metadata.h5', column_names, description)



Model saved to my_model_with_metadata.h5
Metadata added to my_model_with_metadata.h5


In [11]:
# Loading Model Metadata

metadata_path = 'my_model_with_metadata.h5'
column_names, description = loadModelMetadata(metadata_path)
print(f"Column Names: {column_names}")
print(f"Description: {description}")

Column Names: [b'col1', b'col2', b'col3']
Description: This model is trained on data columns col1, col2, col3
